##Dependencies

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=7f943079b1756cab63dac656c11ce380441f4fb42c07170ab88226b0ce67e2de
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 http

In [2]:
!pip install --quiet findspark
import findspark
findspark.init()

In [3]:
import pyspark
import string
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.mllib.util import MLUtils
from pyspark.sql.types import *
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel, Tokenizer, RegexTokenizer, StopWordsRemover

sc = pyspark.SparkContext(appName = "LDA_app")

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

# Data processing
import pandas as pd
import numpy as np

In [4]:
!wget "https://www.dropbox.com/s/brhwpht2tynabnu/data.zip?dl=1" -O data.zip
!unzip -o data.zip
!rm data.zip

--2024-08-16 05:51:05--  https://www.dropbox.com/s/brhwpht2tynabnu/data.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/1qviajugkvttlmlft9oll/data.zip?rlkey=fbs4s3eooi051wqwcxq34pg53&dl=1 [following]
--2024-08-16 05:51:05--  https://www.dropbox.com/scl/fi/1qviajugkvttlmlft9oll/data.zip?rlkey=fbs4s3eooi051wqwcxq34pg53&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5bd0275065b1c4df91052b1dc6.dl.dropboxusercontent.com/cd/0/inline/CYvmoPHLT9nSQ2yLN3ITHLjLv0ESAkMptVefXmizWqc5okY1yz77GDSA2WpQWrL9pCMNP_A1WvmB9SlwiCIZ1RMopl80D0F9QJDFa-ZQoer2f2IJpGLmtMzKxzbFLUR-0-s/file?dl=1# [following]
--2024-08-16 05:51:06--  https://uc5bd0275065b1c4df91052b1dc6.dl.dropboxusercontent.com/cd/0/inline/CYvmoPHLT9nSQ2

In [5]:
# load in stance detection dataset
filepath = "train.csv"
data = pd.read_csv(filepath, sep=",")
data.tail()

,created_at,id,full_text,entities,is_quote_status,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,possibly_sensitive_appealable,lang,text,label
3631,Thu Mar 26 19:50:51 +0000 2020,2895,Help de zorg en bescherm onze meest kwetsbaren...,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,46.0,0.0,False,False,NaN,NaN,nl,Help de zorg en bescherm onze meest kwetsbaren...,EENS
3632,Sun May 24 01:22:45 +0000 2020,2763,Eureka!\r\nAls al de tegenstanders van de #loc...,"{'hashtags': [{'text': 'lockdown', 'indices': ...",False,0.0,0.0,False,False,NaN,NaN,nl,Eureka!\r\nAls al de tegenstanders van de #loc...,EENS
3633,Mon Apr 20 09:28:13 +0000 2020,905,"@AdriaanBeenen Als Rutte echt denkt dat de 1,5...",{'hashtags': [{'text': 'anderhalvemetersamenle...,False,1.0,2.0,False,False,NaN,NaN,nl,"Als Rutte echt denkt dat de 1,5 mtr samenlevi...",ONEENS
3634,Fri Jun 19 21:31:38 +0000 2020,3980,Sinds wanneer is een ziekte strafbaar? (#quara...,"{'hashtags': [{'text': 'quarantaine', 'indices...",False,1.0,0.0,False,False,False,False,nl,Sinds wanneer is een ziekte strafbaar? (#quara...,ONEENS
3635,Wed Jun 10 13:07:44 +0000 2020,235,Raadslid Wil van Soest wil weten waarom Halsem...,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,8.0,0.0,False,False,NaN,NaN,nl,Raadslid Wil van Soest wil weten waarom Halsem...,ANDERS


In [6]:
# # Initialize Spark session
# spark = SparkSession.builder.appName("StanceDetection").getOrCreate()

# # Load the dataset
# filepath = "train.csv"

# # Read CSV file into PySpark DataFrame
# data = spark.read.option("sep", ",").option("header", "true").csv(filepath)

# # Show the first few rows of the DataFrame (equivalent to pd.DataFrame.head())
# data.show(5)


## Text processing

In [7]:
import re

punct = {'💁', '[', '’', '~', '💪', '📚', '🏡', '-', '🐣', '🇺', '”', '̶', '\u200a', ';', '🍕', ' ', '!', '%', ',', '👇', '®', '🌈', '?', '🏽', '=', '💨', '✅', '✔', ')', '|', '‘', '\xa0', '🗽', '&', '🏼', '¿', '…', '🎓', '👉', '❌', '🎧', '👈', '🚂', '+', '🤖', '👎', '→', '¡', '🤔', '️', '👸', '@', '🇸', ':', '“', '•', '🏿', '🏻', '👀', '👏', '—', ']', '✓', '"', '\u200b', '🎤', '\n', '.', '(', '$', '❤', '⬇', '#', '👍', "'", '/', '*', '🏾', '–', '👿'}

punct.remove(' ')  # keep spaces
punct.remove('#')  # keep hashtags
punct.remove('@')  # keep mentions
punct.remove('\'') # keep single quotes (in order to retain I'm, isn't, etc.)

def clean(text):
    temp_text = text.lower()
    temp_text = re.sub(r'https?://\S+', '', temp_text)
    temp_text = re.sub(r'\d+', '0', temp_text)
    temp_text = temp_text.replace('’', '\'')  # some single quotes are slanted, and we want to retain them
    for p in punct:
        temp_text = temp_text.replace(p, ' ')
    temp_text = re.sub(r'\s+', ' ', temp_text)
    cleaned_text = temp_text.strip()
    return cleaned_text

In [8]:
def tokenize(tweet_text):
  tknzr = TweetTokenizer()
  tokens=tknzr.tokenize(tweet_text)
  return tokens

In [9]:
data["cleaned"] = data["text"].apply(clean)
data["words"] = data["cleaned"].apply(tokenize)

In [10]:
# Initialize Spark session
spark = SparkSession.builder.appName("StanceDetection").getOrCreate()
spark_df = spark.createDataFrame(data)
spark_df.show(5)

+--------------------+----+--------------------+--------------------+---------------+-------------+--------------+---------+---------+------------------+-----------------------------+----+--------------------+------+--------------------+--------------------+
|          created_at|  id|           full_text|            entities|is_quote_status|retweet_count|favorite_count|favorited|retweeted|possibly_sensitive|possibly_sensitive_appealable|lang|                text| label|             cleaned|               words|
+--------------------+----+--------------------+--------------------+---------------+-------------+--------------+---------+---------+------------------+-----------------------------+----+--------------------+------+--------------------+--------------------+
|Wed Mar 11 13:06:...|1533|@SBrusje @linnas7...|{'hashtags': [{'t...|          False|          0.0|           1.0|    False|    False|               NaN|                          NaN|  nl|    Dan hebben ze...|ANDERS|dan heb

In [11]:
#remove 20 most occuring documents, documents with non numeric characters, and documents with <= 3 characters
cv_tmp = CountVectorizer(inputCol="words", outputCol="tmp_vectors")
cv_tmp_model = cv_tmp.fit(spark_df)


top20 = list(cv_tmp_model.vocabulary[0:20])
more_then_3_charachters = [word for word in cv_tmp_model.vocabulary if len(word) <= 3]
contains_digits = [word for word in cv_tmp_model.vocabulary if any(char.isdigit() for char in word)]

# Download stopwords if you haven't already
nltk.download('stopwords')

# Get Dutch stopwords as a list
stopwords = stopwords.words('dutch')

#Combine the three stopwords
stopwords = stopwords + top20 + more_then_3_charachters + contains_digits

#Remove stopwords from the tokenized list
remover = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords = stopwords)
wordsDataFrame = remover.transform(spark_df)

#Create a new CountVectorizer model without the stopwords
cv = CountVectorizer(inputCol="filtered", outputCol="vectors")
cvmodel = cv.fit(wordsDataFrame)
df_vect = cvmodel.transform(wordsDataFrame)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.ml.clustering import LDA
from pyspark.ml.linalg import Vectors, VectorUDT

# Initialize Spark session
spark = SparkSession.builder.appName("LDA_example").getOrCreate()

# Example DataFrame (assuming you already have df_vect)
# df_vect = spark.read.csv("your_data.csv", header=True, inferSchema=True)

# Example parseVectors function
def parseVectors(line):
    return (int(line[2]), line[0])  # Modify as necessary to return correct format

# Convert DataFrame to RDD and parse
sparsevector_rdd = df_vect.select('vectors', 'text', 'id').rdd.map(parseVectors)

# Define schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("features", VectorUDT(), True)
])

# Convert RDD to DataFrame
df = spark.createDataFrame(sparsevector_rdd, schema)

# Initialize and train the LDA model
lda = LDA(k=5, maxIter=10, seed=1, featuresCol='features', optimizer='online')
model = lda.fit(df)

# Print the topics in the model
topics = model.describeTopics(maxTermsPerTopic=15).collect()

# Assuming cvmodel is defined and has the vocabulary attribute
# cvmodel = ... (load or define your CountVectorizer model)

for x, row in enumerate(topics):
    print('Topic number: ' + str(x))
    words = row['termIndices']
    weights = row['termWeights']
    for n in range(len(words)):
        print(cvmodel.vocabulary[words[n]] + ' ' + str(weights[n]))


Topic number: 0
mensen 0.008319326916815514
anderhalve 0.0062602265915572336
elkaar 0.005896582071747818
weer 0.0041825075002934675
houd 0.004121815009317227
gaan 0.0028879658252392282
blijf 0.0027956829993769435
moeten 0.0027135327847427207
thuis 0.0026636756851713393
alleen 0.002623671716643883
iedereen 0.0026234541293194058
buiten 0.0025252146255743475
goed 0.0024155903307100713
waar 0.002264940968694007
corona 0.0021445912007319197
Topic number: 1
vanaf 0.00028354046761189814
elkaar 0.000254256304623373
miljoen 0.0002514556022981041
maatschappij 0.0002496479282338025
apart 0.0002460248185941758
goed 0.00023863462194057008
euro 0.00021877171869036244
deelt 0.00021402801117738433
vijf 0.00021016247253925847
onderdrukt 0.0002076505284230324
mits 0.00020756021660438232
vrouw 0.00020705828805122345
zien 0.00020604245333889163
compensatie 0.00020293267794979677
vervoerders 0.000201908703458855
Topic number: 2
anderhalve 0.0008817454351102398
hond 0.0006011425200842562
maurice 0.000459616